In [ ]:
%matplotlib inline

Interlaminar Normal Stresses {#interlaminar_normal_stress_example}
============================

Compute interlaminar normal stresses for layered shells.

This example shows how to enrich the stresses of layered shells by
interlaminar normal stresses, in short INS. INS can be of importance in
thick and curved laminates.

For simple use cases it is preferable to use the composite failure
operator
(`sphx_glr_examples_gallery_examples_1_failure_operator_example.py`{.interpreted-text
role="ref"}) or the composite sampling point operator
(`sphx_glr_examples_gallery_examples_2_sampling_point_example.py`{.interpreted-text
role="ref"}). Note, the INS are computed automatically in these
workflows if required. For instance if a 3D failure criterion is
activated. The
`sphx_glr_examples_gallery_examples_6_filter_composite_data_example.py`{.interpreted-text
role="ref"} example shows how helper functions can be used to obtain
composite result data.

INS are typically not available if layered shell elements are used. The
INS operator recomputes s3 based on the laminate strains, the
geometrical curvature and the lay-up.


Script
======

Load Ansys libraries


In [ ]:
import ansys.dpf.core as dpf

from ansys.dpf.composites import CompositeModel, Spot, get_selected_indices
from ansys.dpf.composites.connect_to_or_start_server import connect_to_or_start_server
from ansys.dpf.composites.enums import Sym3x3TensorComponent
from ansys.dpf.composites.example_helper.example_helper import get_continuous_fiber_example_files
from ansys.dpf.composites.layup_info import AnalysisPlyInfoProvider, get_all_analysis_ply_names

Start Server and upload files


In [ ]:
server = connect_to_or_start_server()
composite_files_on_server = get_continuous_fiber_example_files(server, "ins")

Configure data sources


In [ ]:
composite_model = CompositeModel(composite_files_on_server, server)

Prepare inputs for the INS operator

Rotate to global is False because the post-processing engine expects the
results to be in the element coordinate system (material coordinate
system).


In [ ]:
strain_operator = dpf.Operator("EPEL")
strain_operator.inputs.data_sources(composite_model.data_sources.rst)
strain_operator.inputs.bool_rotate_to_global(False)

stress_operator = dpf.Operator("S")
stress_operator.inputs.data_sources(composite_model.data_sources.rst)
stress_operator.inputs.bool_rotate_to_global(False)

Compute interlaminar normal stresses
====================================

Note: the INS operator stores the stresses in the provided stressed
field.


In [ ]:
composite_model.add_interlaminar_normal_stresses(
    stresses=stress_operator.outputs.fields_container(),
    strains=strain_operator.outputs.fields_container(),
)

Plot s3 stresses
================

Get the first stress field


In [ ]:
stress_field = stress_operator.outputs.fields_container()[0]

Plot max s3 of each element


In [ ]:
s3_component = Sym3x3TensorComponent.tensor33
max_s3_field = dpf.field.Field(location=dpf.locations.elemental, nature=dpf.natures.scalar)
with max_s3_field.as_local_field() as local_max_s3_field:
    element_ids = stress_field.scoping.ids
    for element_id in element_ids:
        stress_data = stress_field.get_entity_data_by_id(element_id)
        element_info = composite_model.get_element_info(element_id)
        assert element_info is not None
        # select all stresses from bottom to top of node 0
        selected_indices = get_selected_indices(element_info, nodes=[0])

        # order is bottom, top, mid
        s3 = stress_data[selected_indices, s3_component.value]

        local_max_s3_field.append([max(s3)], element_id)

composite_model.get_mesh().plot(max_s3_field)

Plot s3 at the mid-plane of a certain ply


In [ ]:
analysis_ply_names = get_all_analysis_ply_names(composite_model.get_mesh())
selected_ply = "P3L1__Ply.1"

ply_info_provider = AnalysisPlyInfoProvider(composite_model.get_mesh(), selected_ply)
p8l1_ply_s3_field = dpf.field.Field(location=dpf.locations.elemental, nature=dpf.natures.scalar)
with p8l1_ply_s3_field.as_local_field() as p8l1_ply_s3_field:
    element_ids = ply_info_provider.ply_element_ids()
    for element_id in element_ids:
        stress_data = stress_field.get_entity_data_by_id(element_id)
        element_info = composite_model.get_element_info(element_id)
        assert element_info is not None
        # select all stresses from bottom to top of node 0
        layer_index = ply_info_provider.get_layer_index_by_element_id(element_id)
        selected_indices = get_selected_indices(
            element_info, layers=[layer_index], nodes=[0], spots=[Spot.MIDDLE]
        )

        # order is bottom, top, mid
        s3 = stress_data[selected_indices, s3_component.value]

        p8l1_ply_s3_field.append(s3, element_id)

composite_model.get_mesh().plot(p8l1_ply_s3_field)